# 🔬 Deep Dive: Python Object Model & Memory Optimization
This notebook provides a comprehensive exploration of Python's object internals, reference counting, memory layout, garbage collection, interning, and how to optimize memory using tools like `__slots__`, `weakref`, and diagnostics like `objgraph` and `tracemalloc`.

⚠️ This notebook assumes you're comfortable with Python classes, memory models, and want to go deep into CPython behavior.

## 1️⃣ Reference Counting & Garbage Collection
Every object in CPython has a reference count. When it reaches zero, the object is deallocated.

In [ ]:

import sys
import gc

x = []
y = x
print("Reference count of x:", sys.getrefcount(x))  # sys.getrefcount adds +1 itself

# Create cyclic ref
x.append(y)
del x
del y

print("Before GC: Garbage count =", gc.collect())  # Force collection


### 🔄 Generational Garbage Collection
CPython uses 3 generations (0, 1, 2). Objects that survive longer move up.

In [ ]:

for gen in range(3):
    print(f"Generation {gen} threshold:", gc.get_threshold()[gen])
    print(f"Objects in gen {gen}:", len(gc.get_objects()))


## 👻 Weak References and Object Lifetimes

In [ ]:

import weakref

class Data:
    def __del__(self):
        print("Data object deleted")

d = Data()
r = weakref.ref(d)
print("Weakref points to:", r())

del d
print("Weakref after deletion:", r())


## 🔢 Small Integer and String Interning
Python caches small integers in [-5, 256] and interns certain strings.

In [ ]:

import sys
a = 100
b = 100
print("a is b:", a is b)

x = 1000
y = 1000
print("x is y:", x is y)

s1 = sys.intern("very_common_string")
s2 = sys.intern("very_common_string")
print("s1 is s2 (interned):", s1 is s2)


## 🧱 Object Headers and Memory Layout

In [ ]:

import sys

class Demo:
    def __init__(self):
        self.a = 1
        self.b = 2

obj = Demo()
print("Size with __dict__:", sys.getsizeof(obj))
print("Attributes:", obj.__dict__)


## 🧳 Using `__slots__` for Memory Efficiency
Slots eliminate per-instance `__dict__` and save memory when many objects are created.

In [ ]:

class Slotted:
    __slots__ = ['a', 'b']
    def __init__(self):
        self.a = 1
        self.b = 2

class Normal:
    def __init__(self):
        self.a = 1
        self.b = 2

import sys

normal_objs = [Normal() for _ in range(10000)]
slotted_objs = [Slotted() for _ in range(10000)]

print("Normal one size:", sys.getsizeof(normal_objs[0]))
print("Slotted one size:", sys.getsizeof(slotted_objs[0]))


## 🧠 Memory Diagnostics: `tracemalloc` and `objgraph`
Use these tools to detect memory leaks and object allocation patterns.

In [ ]:

import tracemalloc

tracemalloc.start()

objs = [Normal() for _ in range(50000)]
snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno')

print("Top 5 memory lines:")
for stat in top_stats[:5]:
    print(stat)


### 🔍 Visualizing with `objgraph`
(You must install `objgraph` via pip if missing)

In [ ]:

# !pip install objgraph
import objgraph

class Tree:
    def __init__(self, child=None):
        self.child = child

t = Tree(Tree(Tree()))
objgraph.show_refs([t], filename='tree_refs.png')  # generates a PNG visualization


## ✅ Best Practices Recap


- Use `__slots__` to reduce memory for frequently created classes
- Monitor allocations with `tracemalloc` during development
- Use `gc.collect()` sparingly to clean cyclic structures
- Avoid unnecessary references to reduce GC workload
- Use `weakref` for caches, graphs, plugins to avoid memory leaks
